# Hyperspy Tutorial

## EELS analysis of perovskite oxides

This tutorial shows the various functionalities in HyperSpy which is used to analyse Electron Energy Loss Spectroscopy data, using EELS datasets from perovskite oxide heterostructure.

It assumes some knowledge on how to use HyperSpy, like loading datasets and how the basic signals work.

This notebook requires:

HyperSpy 0.8.5 and all dependencies 

## Author

7/6/2016 Magnus Nord - Developed for HyperSpy workshop at Scandem conference 2016

## Table of contents

1. <a href='#spec_and_data'> Specimen & Data</a>
2. <a href='#simple_quant'> Simple quantification</a>
3. <a href='#curve_fitting_quant'> Curve fitting quantification</a>
4. <a href='#fine_structure_analysis'> Fine structure analysis</a>
5. <a href='#fine_structure_ok'> Fine structure oxygen-K edge</a>

# <a id='spec_and_data'></a>1. Specimen & Data

This notebook was used for the HyperSpy workshop at the Norwegian University of Science and Technology for the Scandem 2016 conference, 7 June 2016.

The data was acquired on a Jeol ARM200cF using a Gatan Quantum ER with DualEELS capabilites.

The data itself is from La0.7Sr0.3MnO3 thin films deposited on SrTiO3. In the Fine Structure example parts of the film has been exposed to a very long electron beam exposure, inducing oxygen vacancies.

The datasets has been binned to reduce the file size and processing time.

# <a id='simple_quant'></a> 2. Simple quantification


Firstly we use some IPython magic to import the right plotting libraries, then import HyperSpy

In [1]:
%matplotlib qt4

In [2]:
import hyperspy.api as hs

/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/io_plugins/tiff.py:31: UserWarning: Failed to import the optional scikit image package. Loading of some compressed images will be slow.

  "Failed to import the optional scikit image package. "


First we take a look at an EELS line scan across an La0.7Sr0.3MnO3/SrTiO3 thin film. The core loss data has several peaks: Ti-L23, O-K, Mn-L23 and La-M54. We can navigate the line scan using the navigation window, and by moving the red line.

In [ ]:
s = hs.load("datasets/LSMO_STO_linescan.hdf5")

In [ ]:
s.plot()

Now we can quantifiy the first edge (Ti-L23). Firstly by removing the background, then cropping the signal to only include the Ti-L23 edge

In [ ]:
s.remove_background()

In [ ]:
s.crop_spectrum()

Note, both these functions replaced the old signals s.

We can also do this using only the command line, which does not replace the original signal:

In [ ]:
s = hs.load("datasets/LSMO_STO_linescan.hdf5")

In [ ]:
s1 = s.remove_background(signal_range=(405.,448.))

In [ ]:
s2 = s1.isig[448.:480.]

Then we can sum the energy loss axis to get the relative Ti-content

In [ ]:
s_ti = s2.sum("Energy loss")

In [ ]:
s_ti.plot()

# <a id='curve_fitting_quant'></a> 3. Curve fitting quantification

Now, lets do some more advanced quantification using HyperSpy's extensive modelling framework. Firstly we load the low loss and core loss spectra.

Firstly we'll have to tell HyperSpy where to find the Hartree-Slater cross section files, since they are not included in HyperSpy. Go to the "EELS" tab, then "Model", then set "GOS directory" to the "H-S GOS Tables" folder

In [ ]:
hs.preferences.gui()

In [140]:
s_ll = hs.load("datasets/LSMO_STO_linescan_low_loss.hdf5")

/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


In [141]:
s = hs.load("datasets/LSMO_STO_linescan.hdf5")

/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


Here, the metadata has been populated with some of the experimental parameters:

In [142]:
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       └── collection_angle = 33.100000000000001
│       ├── beam_energy = 200.0
│       ├── convergence_angle = 27.100000000000001
│       └── dwell_time = 0.49990557338919173
├── General
│   ├── original_filename = LSMO_STO_linescan.dm3
│   └── title = EELS Spectrum Image (high-loss)
└── Signal
    ├── binned = True
    ├── record_by = spectrum
    ├── signal_origin = 
    └── signal_type = EELS

Firstly, we make sure the energy scale is properly aligned by using the zero loss peak in the low loss spectrum.

In [143]:
s_ll.align_zero_loss_peak(subpixel=True, also_align=[s])


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-1.000
std:	0.000

min:	-1.000
Q1:	-1.000
median:	-1.000
Q3:	-1.000
max:	-1.000
 calculating 100% |#############################################| ETA:  00:00:00 

 calculating 100% |#############################################| ETA:  00:00:00 

 calculating 100% |#############################################| ETA:  00:00:00 

 calculating 100% |#############################################| ETA:  00:00:00 

 calculating 100% |#############################################| ETA:  00:00:00 



/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:132: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


We have to add the elements which is present in the sample to `s`

In [144]:
s.add_elements(('Mn','O','Ti','La'))

Then we make a model out of the core loss spectrum. The low loss spectrum is convolved with the model, which means plural scattering is automatically taken into account. In addition this leads to better fits.

In [145]:
m = s.create_model(ll=s_ll)

/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/models/eelsmodel.py:86: VisibleDeprecationWarning: Adding "background" to the user namespace. This feature will be removed in Hyperspy 1.0.
  VisibleDeprecationWarning)
/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/signals.py:132: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)
/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/models/eelsmodel.py:188: VisibleDeprecationWarning: Adding "Ti_L3" to the user namespace. This feature will be removed in Hyperspy 1.0.
  VisibleDeprecationWarning)
/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/models/eelsmodel.py:194: VisibleDeprecationWarning: Add


Hartree-Slater GOS
	Element:  Ti
	Subshell:  L3
	Onset Energy =  456.0

Hartree-Slater GOS
	Element:  Ti
	Subshell:  L2
	Onset Energy =  462.0

Hartree-Slater GOS
	Element:  Ti
	Subshell:  L1
	Onset Energy =  564.0

Hartree-Slater GOS
	Element:  O
	Subshell:  K
	Onset Energy =  532.0

Hartree-Slater GOS
	Element:  Mn
	Subshell:  L3
	Onset Energy =  640.0

Hartree-Slater GOS
	Element:  Mn
	Subshell:  L2
	Onset Energy =  651.0

Hartree-Slater GOS
	Element:  Mn
	Subshell:  L1
	Onset Energy =  769.0

Hartree-Slater GOS
	Element:  La
	Subshell:  M5
	Onset Energy =  832.0

Hartree-Slater GOS
	Element:  La
	Subshell:  M4
	Onset Energy =  849.0


/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/models/eelsmodel.py:227: VisibleDeprecationWarning: Adding "La_M4" to the user namespace. This feature will be removed in Hyperspy 1.0.
  VisibleDeprecationWarning)


The model new consist of many different EELSCLEdge components, including a component for the plasmon background

In [146]:
m.components

   # |            Attribute Name |            Component Name |            Component Type
---- | ------------------------- | ------------------------- | -------------------------
   0 |                background |                background |                  PowerLaw
   1 |                     Ti_L3 |                     Ti_L3 |                EELSCLEdge
   2 |                     Ti_L2 |                     Ti_L2 |                EELSCLEdge
   3 |                     Ti_L1 |                     Ti_L1 |                EELSCLEdge
   4 |                       O_K |                       O_K |                EELSCLEdge
   5 |                     Mn_L3 |                     Mn_L3 |                EELSCLEdge
   6 |                     Mn_L2 |                     Mn_L2 |                EELSCLEdge
   7 |                     Mn_L1 |                     Mn_L1 |                EELSCLEdge
   8 |                     La_M5 |                     La_M5 |                EELSCLEdge
   9 |               

We can fit the model to the experimental data by using the `multifit` function, with the `smart` fitting. Which is fits in a way optimized for EELS data, by fitting from the lowest to the highest energy losses.

In [147]:
m.multifit(kind='smart')

 calculating 100% |#############################################| ETA:  00:00:00 



In [159]:
m.plot()

We can check the error of the fitting

In [160]:
edges = ("Ti_L3", "La_M5", "Mn_L3","O_K")

In [161]:
hs.plot.plot_spectra([m[edge].intensity.as_signal("std") for edge in edges], legend=edges)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:894: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


This fitted mostly ok, but it is still not very good. Firstly we can move the Hartree-Slater onsets interactively

In [ ]:
m.enable_adjust_position()

Or manually, by directly changing the parameters within the Hartree-Slater edges. The parameter is called onset_energy

In [149]:
m.components.O_K.onset_energy.value = 528

However, to change it for all the probe positions we have to use assign_current_value_to_all()

In [150]:
m.components.O_K.onset_energy.assign_current_value_to_all()

We repeat this for the Manganese edges. Since this is an L-edge, there are 3 different ones. However, we only have to set the Mn-L3: the L2 and L1 is a set to an energy relative to the L3.

In [151]:
m.components.Mn_L3.onset_energy.value

640.0

In [152]:
m.components.Mn_L2.onset_energy.value

651.0

In [153]:
m.components.Mn_L3.onset_energy.value = 638.5

In [154]:
m.components.Mn_L2.onset_energy.value 

649.5

In [155]:
m.components.Mn_L3.onset_energy.assign_current_value_to_all()

This is due to the fine structure not currently taken into account by the model. To get a good fit, we can either not fit to the fine structure regions, or model them somehow.
The easiest way is defining certain regions as fine structure:

In [156]:
m.enable_fine_structure()

Automatically changing the fine structure width of edge 1 from 30.0 eV to 4.0 eV to avoid conflicts with edge number 2
Automatically changing the fine structure width of edge 5 from 30.0 eV to 9.0 eV to avoid conflicts with edge number 6
Automatically changing the fine structure width of edge 8 from 30.0 eV to 15.0 eV to avoid conflicts with edge number 9


This will produce a much better fit, but will be much slower (~2 minutes).

In [157]:
m.multifit(kind='smart')

 calculating   0% |                                             | ETA:  --:--:-- 

/home/magnunor/Documents/HyperSpy_project/HyperSpy/hyperspy/model.py:1254: RuntimeWarning: invalid value encountered in sqrt
  self.p_std = np.sqrt(np.diag(pcov))


 calculating 100% |#############################################| ETA:  00:00:00 



Now the fit is much better, due to the model taking into account the fine structure.

In [134]:
m.plot()

Now we can can have a look at the relative intensity from the individual EELS-edges using plot_spectra

In [122]:
edges = ("Ti_L3", "La_M5", "Mn_L3","O_K")

In [123]:
hs.plot.plot_spectra([m[edge].intensity.as_signal() for edge in edges], legend=edges)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:894: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


While the fitting looks nice, we can clearly improve this. Firstly the intensites are negative where it should be zero. Secondly, the fine structure regions can be fine tuned. Especially the Mn-L1 fine structure window can be reduced

In [124]:
m.components.Mn_L1.fine_structure_width = 15

To avoid the negative values we use bounded fitting, where we can constrain the parametern values between certain values. The bmin and bmax properties in the parameters are used for this.

In [125]:
m.components.Mn_L3.intensity.bmin = 0.0

In [126]:
m.components.La_M5.intensity.bmin = 0.0

In [127]:
m.components.Ti_L3.intensity.bmin = 0.0

In [128]:
m.components.O_K.intensity.bmin = 0.0

Then we use the fitter='mpfit' and bounded=True arguments when running the multifitter

In [135]:
m.disable_fine_structure()

In [136]:
m.ensure_parameters_in_bounds()

In [137]:
m.multifit(kind='smart', fitter='mpfit', bounded=True)

 calculating 100% |#############################################| ETA:  00:00:00 



In [138]:
m.plot()

In [133]:
hs.plot.plot_spectra([m[edge].intensity.as_signal() for edge in edges], legend=edges)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:894: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# <a id='fine_structure_analysis'></a> 4. Fine structure analysis

Here we take a look at a linescan from a La0.7Sr0.3MnO3 thin film, where parts of the film has been bombarded with the electron beam for an extended time.

In [ ]:
s = hs.load("datasets/LSMO_linescan.hdf5")

Using the moving the red line in the there is clearly something going on in the middle on both the oxygen and the manganese edges. In addition, there are some thickness changes during the line scan.

In [ ]:
s.plot()

Using the low loss signal, we make sure the energy scale is properly calibrated

In [ ]:
s_ll = hs.load("datasets/LSMO_linescan_low_loss.hdf5")

In [ ]:
s_ll.plot()

The zero loss peak is not well aligned at 0 eV energy loss, so we should align it and the core loss

In [ ]:
s_ll.align_zero_loss_peak(subpixel=True, also_align=[s])

Now the zero loss peak has been shifted to 0 energy loss, and likewise the core loss spectrum `s` has also been aligned

In [ ]:
s_ll.plot()

We can also calculate the relative thickness using the low loss. We'll have to specify the end of the zero loss beam, which for cold field emissions guns 3.0 eV seems to work well.

In [ ]:
s_thickness = s_ll.estimate_thickness(threshold=3.0)

This gives the relative thickness and, as expected, there is an increase towards the end of the line scan

In [ ]:
s_thickness.plot()

# <a id='fine_structure_ok'></a> 5. Fine structure: oxygen K-edge
Lets take a closer look at the oxygen-K edge, firstly by removing the plasmon background, then cropping the spectrum to only include the oxygen-K edge. Note: this will overwrite the `s` spectrum with the cropped one. 

In [ ]:
s.remove_background()

In [ ]:
s.crop_spectrum()

This makes it much easier to compare the different positions. Pressing 'e' with the spectrum window highlighted gives a second spectrum picker, which can be moved independently of the first one

We can then do fourier ratio deconvolution to remove the effects of plural scattering

In [ ]:
s_deconvolved = s.fourier_ratio_deconvolution(s_ll)

In [ ]:
s_deconvolved.plot()

### Fine structure modelling

Having had a qualitativily look at the data, we can try to quantify some of these changes. We do this by making making a model of the oxygen-K edge signal. As we've already removed the background, we set `auto_background=False`.

In [ ]:
m = s.create_model(ll=s_ll, auto_background=False)

So currently, the model does not contain any components

In [ ]:
m.components

We can try to model some of the fine structure with Gaussians

In [ ]:
g1 = hs.model.components.Gaussian()

In [ ]:
m.append(g1)

This added the gaussian component to the model

In [ ]:
m.components

Then we can fit this Gaussian to the largest of the O-K peaks by dragging a span over the peak between 528 and 533 eV. Run it first with the "Only Current" option ticked, then run it without to fit the whole dataset

In [ ]:
m.fit_component(g1)

Having fitted the Gaussian to the experimental data, we can plot how the Gaussian three parameters change over the line scan: A, sigma and centre. The A changes quite a bit, which is probably (among others) related to thickness changes. However, there are clear changes in the sigma parameter in the region with the electron beam damage

In [ ]:
g1.plot()

Using the same method we can also fit the second largest peak between 535 and 541 eV

In [ ]:
g2 = hs.model.components.Gaussian()

In [ ]:
m.append(g2)

In [ ]:
m.fit_component(g2)

However, this time the final fit does not look very good. This is due to the two components being fitted independently of eachother. We should fit both of them at the same time. Firstly, we have to set the `signal_range` which is where the model will fit to the experimental data. Here we select the region spanning the two major peaks (528-541 eV)

In [ ]:
m.set_signal_range()

In [ ]:
m.multifit()

After fitting, we reset the signal range so we can see the full range of the signal

In [ ]:
m.reset_signal_range()

In [ ]:
m.plot()

Lastly, we can fit the small "pre-peak" as well. First we "lock" the two Gaussian we have already fitted.

In [ ]:
g1.set_parameters_not_free()

In [ ]:
g2.set_parameters_not_free()

Then we add another Gaussian, and fit it using `fit_component` between 522 and 527 eV

In [ ]:
g3 = hs.model.components.Gaussian()

In [ ]:
m.append(g3)

In [ ]:
m.fit_component(g3)

Then we set the signal range to cover all the three peaks, from 520 eV to 541 eV

In [ ]:
m.set_signal_range()

In [ ]:
m.multifit()

This fits all the three components to the experimental data, which hopefully gives a good fit

In [ ]:
m.reset_signal_range()

In [ ]:
m.plot()

Then we can save the sigma of the different components as signals:

In [ ]:
s_g1s = g1.sigma.as_signal()

In [ ]:
s_g2s = g2.sigma.as_signal()

In [ ]:
s_g3s = g3.sigma.as_signal()